# Utilização do algoritmo TOPSIS

In [1]:
def topsis_equal_weights(dataframe, pos_cols, neg_cols ):
    ''' O dataframe original tem que está no formato
        onde a primeira coluna são os tickets e a segunda 
        os anos ou vice-versa'''
    
    
    # Normaliza o dataframe por coluna
    df_norm = dataframe.copy()
    
    ## Retira colunas de anos e ticket e normaliza cada uma
    for col in df_norm.columns[2:]:
        df_norm[col] = df_norm[col] / np.sqrt(np.sum(df_norm[col]**2))
        
    # Cria coluna de pesos sendo todas iguais    
    weights = np.full((df_norm.iloc[:,2:].shape[1], 1), 1/df_norm.iloc[:,2:].shape[1])
    
    ## Multiplica a matriz normalizada pelos pesos
    df_norm_w = df_norm.copy()
    df_norm_w.iloc[:, 2:] =  df_norm_w.iloc[:, 2:]*weights.ravel()
    
    
    # Best option
    max_positive = np.max(df_norm_w[pos_cols], axis=0)
    min_negative = np.min(df_norm_w[neg_cols], axis=0)

    #Worst option
    min_positive = np.min(df_norm_w[pos_cols], axis=0)
    max_negative = np.max(df_norm_w[neg_cols], axis=0)
    
    # Calculate d_max
    d_max_dif_pos = np.sum((df_norm_w[pos_cols] - max_positive)**2, axis =1)
    d_max_dif_neg = np.sum((df_norm_w[neg_cols] - min_negative)**2, axis =1)
    d_max = np.sqrt(d_max_dif_pos + d_max_dif_neg)

    # Calculate d_min
    d_min_dif_pos = np.sum((df_norm_w[pos_cols] - min_positive)**2, axis =1)
    d_min_dif_neg = np.sum((df_norm_w[neg_cols] - max_negative)**2, axis =1)
    d_min = np.sqrt(d_min_dif_pos + d_min_dif_neg)
    
    # Dataframe final
    df_final = dataframe.iloc[:,:2]
    df_final['S*'] = d_max
    df_final['S-'] = d_min
    df_final['C*'] = d_min/(d_max+d_min)
    
    # Agroupa por tickets
    df_grouped = df_final.groupby('ticket')[['S*', 'S-', 'C*']].mean().reset_index()
    df_grouped = df_grouped.sort_values(by="C*", ascending=False).reset_index(drop = True)
    
    return df_grouped

In [ ]:
def topsis(dataframe, pos_cols, neg_cols, pesos_df):
    ''' O dataframe original tem que está no formato
        onde a primeira coluna são os tickets e a segunda 
        os anos ou vice-versa'''
    # Normaliza o dataframe por coluna
    df_norm = dataframe.copy()
    
    ## Retira colunas de anos e ticket e normaliza cada uma
    for col in df_norm.columns[2:]:
        df_norm[col] = df_norm[col] / np.sqrt(np.sum(df_norm[col]**2))
        
    # Cria coluna de pesos sendo todas iguais    
    weights = pesos_df.transpose().values
    
    ## Multiplica a matriz normalizada pelos pesos
    df_norm_w = df_norm.copy()
    df_norm_w.iloc[:, 2:] =  df_norm_w.iloc[:, 2:]*weights.ravel()
    
    
    # Best option
    max_positive = np.max(df_norm_w[pos_cols], axis=0)
    min_negative = np.min(df_norm_w[neg_cols], axis=0)

    #Worst option
    min_positive = np.min(df_norm_w[pos_cols], axis=0)
    max_negative = np.max(df_norm_w[neg_cols], axis=0)
    
    # Calculate d_max
    d_max_dif_pos = np.sum((df_norm_w[pos_cols] - max_positive)**2, axis =1)
    d_max_dif_neg = np.sum((df_norm_w[neg_cols] - min_negative)**2, axis =1)
    d_max = np.sqrt(d_max_dif_pos + d_max_dif_neg)

    # Calculate d_min
    d_min_dif_pos = np.sum((df_norm_w[pos_cols] - min_positive)**2, axis =1)
    d_min_dif_neg = np.sum((df_norm_w[neg_cols] - max_negative)**2, axis =1)
    d_min = np.sqrt(d_min_dif_pos + d_min_dif_neg)
    
    # Dataframe final
    df_final = dataframe.iloc[:,:2]
    df_final['S*'] = d_max
    df_final['S-'] = d_min
    df_final['C*'] = d_min/(d_max+d_min)
    
    # Agroupa por tickets
    df_grouped = df_final.groupby('ticket')[['S*', 'S-', 'C*']].mean().reset_index()
    df_grouped = df_grouped.sort_values(by="C*", ascending=False).reset_index(drop = True)
    
    return df_grouped

# Calculate topsis para acoes

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

w_acoes = pd.read_csv("acoes_b3_weights.csv")
df_acoes = pd.read_csv("acoes_b3_stinvest_tratado.csv")

In [ ]:
pos_cols = ['dy%', 'vpa', 
            'lpa', 'patrimonio_ativo', 
            'liquidezcorrente', 'margembruta%', 
            'margemebitda%', 'margemebit%', 
            'margemliquida%', 'roe%', 
            'roa%', 'roic%', 
            'giro_ativos','receitas_cagr5%', 
            'lucros_cagr5%']

neg_cols = ['p_l', 'peg_Ratio', 
            'p_vp', 'ev_ebitda',
            'ev_ebit', 'p_ebita', 
            'p_ebit', 'p_ativo', 
            'p_sr','p_capitlgiro', 
            'p_ativocirculante', 'dividaliquida_patrimonioliquido',
            'dividaliquida_ebitda', 'dividaliquida_ebit',
            'passivo_ativo']

In [ ]:
best_acoes = topsis(df_acoes, pos_cols, neg_cols, w_acoes)
best_acoes.to_csv('melhores_acoes_topsis.csv', index=False)

# Calculate topsis para bdrs

In [ ]:
w_bdrs = pd.read_csv("bdrs_weights.csv")
df_bdrs = pd.read_csv("bdrs_b3_stinvest_tratado.csv")

pos_cols = ['dy%', 'vpa', 
            'lpa', 'patrimonio_ativo', 
            'liquidezcorrente', 'margembruta%', 
            'margemebitda%', 'margemebit%', 
            'margemliquida%', 'roe%', 
            'roa%', 'roic%', 
            'giro_ativos','receitas_cagr5%', 
            'lucros_cagr5%']

neg_cols = ['p_l', 'peg_Ratio', 
            'p_vp', 'ev_ebitda',
            'ev_ebit', 'p_ebita', 
            'p_ebit', 'p_ativo', 
            'p_sr','p_capitlgiro', 
            'p_ativocirculante', 'dividaliquida_patrimonioliquido',
            'dividaliquida_ebitda', 'dividaliquida_ebit',
            'passivo_ativo']

best_bdrs = topsis(df_bdrs, pos_cols, neg_cols, w_bdrs)
best_bdrs.to_csv('melhores_acoes_topsis.csv', index=False)

# Calculate topsis para FIIs

In [ ]:
w_fiis = pd.read_csv("FII_weights.csv")
df_fiis = pd.read_csv("FII_b3_stinvest_tratado.csv")

# Algumas colunas não puxam info do statusinvest
# Seleciono todas as colunas que tem alguma info e tem efeito positivo ou negativo no FII
pos_cols = ['numero cotistas', 'numero cotas emitidas', 'Ativos - (R$)',
            'patrimonio liquido - (R$)', 'Valor patrimonial cota - (R$)',
            'Disponibilidades - (R$)', 'titulos publicos - (R$)', 'Titulos privados - (R$)',
            'Total investido - (R$)', 'Bens e direitos imoveis - (R$)', 'Terrenos - (R$)',
            'Imoveis renda acabados - (R$)', 'Imoveis renda construcao - (R$)',
            'Imoveis venda acabados - (R$)', 'Outros bens e direitos - (R$)',
            'acoes - (R$)', 'Fundo de Investimento Imobiliario (FII) - (R$)',
            'Acoes de Sociedades cujo unico proposito se enquadra entre as atividades permitidas aos FII - (R$)',
            'Cotas de Sociedades que se enquadre entre as atividades permitidas aos FII - (R$)',
            'CRI - (R$)', 'LCI - (R$)', 
            'Valores a Receber - (R$)', 'Contas a Receber por alugueis - (R$)', 
            'Contas a Receber por Venda de Imoveis - (R$)', 'Outros Valores a Receber - (R$)',
            'Rendimentos a distribuir - (R$)']

neg_cols = ['Total necessidade liquidez - (R$)', 'Valores a pagar - (R$)',
            'Taxa de administracao a pagar - (R$)', 'Taxa de performance a pagar - (R$)',
            'obrigacoes por aquisicoes de imoveis - (R$)', 'Obrigacoes por securitizacoes de imoveis - (R$)',
            'Provisoes e contigencias - (R$)', 'Outros valores a pagar - (R$)']

df_fiis_selecionados = df_fiis.loc[:, ['Anos'] + ['Ticket'] + pos_cols + neg_cols]
df_fiis_selecionados.rename(columns={'Ticket': 'ticket'}, inplace = True)

best_fiis = topsis(df_fiis_selecionados, pos_cols, neg_cols, w_fiis)
best_fiis.to_csv('melhores_FIIs_topsis.csv', index=False)